In [42]:
#%pylab inline
import seaborn as sns
import urllib2
#import urllib3  as urllib2
#import urllib.request  as urllib2
import pandas as pd
import re
import operator
from datetime import datetime, date, time
from bs4 import BeautifulSoup
import numpy as np
import itertools
import math
import time
import seaborn as sns
from matplotlib import pyplot
sns.set(style="whitegrid")
from IPython.core.display import display, HTML
import csv
from trueskill import Rating, quality_1vs1, rate_1vs1, quality, rate, global_env, choose_backend,  backends, setup
cdf, pdf, ppf = choose_backend('mpmath')
setup(backend='mpmath')

display(HTML("<style>.container { width:90% !important; }</style>"))
## Configuration Settings
match_start_id = 1
match_end_id = 1900
player_start_id = 1
player_end_id = 190

In [ ]:
#initialize ratings
df_d1_players = df_competition_raw[df_competition_raw["division"] == 'DI'][['first_name', 'last_name', 'college']].drop_duplicates()
df_d1_players['first_name'].sort_values().unique()
df_d1_players['first_name'] = df_d1_players['first_name'].apply(lambda x: x.strip())
df_d1_players = df_d1_players[df_d1_players['first_name'] != 'Forfeit']




In [26]:
#!pip install trueskill
#!pip install mpmath
setup(backend='mpmath')

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=10.0%, backend='mpmath')

# Scrape and Parse URL

In [4]:
## SOUP ATTRIBUTES
def make_soup(url, _id):
    quote_page = url+str(_id)  # specify the url
    page = urllib2.urlopen(quote_page)  # query the website and return the html to the variable ‘page’
    return BeautifulSoup(page, 'html.parser')

def get_header(_soup, _remove, _type):
    h = _soup.find('h1').text.strip().replace(_remove,'')
    if _type == 'event':
        event_date = datetime.datetime.strptime(h, '%A, %B %d, %Y')
        h = {'datetime': event_date.strftime('%Y%m%d'),'day': event_date.strftime('%A')}
        h = h['datetime']
    return h

def get_tag(_soup, _remove, _tag):
    t = _soup.find(_tag).text.strip().replace(_remove,'')
    return t

def get_table_data(_soup):
    name_box = _soup.find('table')
    name_rows = name_box.find_all('tr') 
    strip = [a.text.strip().replace('\r\n','') for a in name_rows]
    strip = [a.split('\n') for a in strip]
    return strip

def scrape_url(_url, _id):
    soup = make_soup(_url, _id)
    table = get_table_data(soup)
    return soup, table

# Teams

## Teams > Scrape Team General Team Info

In [ ]:
def scrape_team_info(division='all'):
    url = 'http://collegebeachvb.com/colleges'
    id_ = ''
    table = get_table_data(make_soup(url, id_))

    my_labels = table[0]
    my_results = table[1:]
    df_ = pd.DataFrame.from_records(my_results,columns=my_labels) 

    if division != 'all':
        df_ = df_[df_.Division == division][['College', 'Team Name', 'Location', 'Division']]
    else:
        df_ = df_[['College', 'Team Name', 'Location', 'Division']]
    df_ = pd.read_csv('beachvb_teams.csv').iloc[:,1:]
    df_['College'] = df_['College'].apply(lambda x: x.strip())
    df_['Team Name'] = df_['Team Name'].apply(lambda x: x.strip())
    df_['Division'] = df_['Division'].apply(lambda x: x.strip())

    return df_

df_teams = scrape_team_info()
df_teams.to_csv("beachvb_teams.csv", encoding='utf-8')

# Players
## Players > Scrape Player Info

In [ ]:
def scrape_player_info(start= player_start_id, end= player_end_id):
    url = 'http://collegebeachvb.com/players/collegeplayers/'
    d = {}
    temp = None
    for i in range(start, end):
        print(i,)
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('College Players for ','')
        if (len(table) > 1) :
            d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
            d[i]['Team'] = header.strip()
            d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
    temp = pd.concat([d[k] for k in d.keys()], axis=0)
    temp = temp[~temp['Name'].str.contains('orfeit')]
    temp = temp.drop(temp.index[temp['Name'].duplicated(keep='first')])

    return temp

df_players = scrape_player_info()
df_players.to_csv("beachvb_players.csv", encoding='utf-8'

#df_players = pd.read_csv('Data/beachvb_players.csv').iloc[:,1:]


In [7]:
def write_frames_to_csv(): 

    df_teams.to_csv("beachvb_teams.csv", encoding='utf-8')
    df_matchups.to_csv("beachvb_matchups.csv", encoding='utf-8')
    df_player_rating.to_csv("beachvb_playerrating.csv", encoding='utf-8')
    df_2018_d1_matchups.to_csv("beachvb_2018d1_matchups.csv", encoding='utf-8')
    df_2018_matchups.to_csv("beachvb_2018_matchups.csv", encoding='utf-8')
    df_2018_d1p1_matchups.to_csv("beachvb_2018d1p1_matchups.csv", encoding='utf-8')
    df_2018_d1p2_matchups.to_csv("beachvb_2018d1p2_matchups.csv", encoding='utf-8')
    df_2018_d1p3_matchups.to_csv("beachvb_2018d1p3_matchups.csv", encoding='utf-8')
    df_2018_d1p4_matchups.to_csv("beachvb_2018d1p4_matchups.csv", encoding='utf-8')
    df_2018_d1p5_matchups.to_csv("beachvb_2018d1p5_matchups.csv", encoding='utf-8')
    keys = s[0].keys()
    with open('matches_dict.csv', 'wb') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(s)

# Matchups
## Matchups > Scrape Matchup Info

In [ ]:

def parse_matchups(url, d, h):
    table = scrape_url(url, i)[1]
    soup = scrape_url(url, i)[0]

    header = soup.find('h1').text.strip().replace('Competition on ','')
    header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')

    d[i] = table
    h[i] = header
    return d,h
    
    
def scrape_matchups(start=match_start_id, end=match_end_id):
    url = 'http://collegebeachvb.com/scores/competition/'
    d = {}
    h = {}
    for i in range(start,end):
        print(i)
        try:
            parse_matchups(url, d, h)
        except:
            print("need to wait....")
            parse_matchups(url, d, h)
    return d, h


df_matchups = scrape_matchups(start=match_start_id, end=match_end_id)

#df_matchups = pd.read_csv('Data/beachvb_matchups.csv').iloc[:,1:]
#for i in range(1,7):
#    df_matchups.iloc[:,i] = df_matchups.iloc[:,i].apply(lambda x: x.strip())
#df_matchups = df_matchups.drop_duplicates()
#df_matchups.count()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
df_matchups



# Matchups
## Matchups > Select Matchups

In [9]:
def select_matchups(startdate, enddate, df=df_matchups, division='DI'):
    temp = df[df['Event Date'].astype(int) >  startdate]
    temp = temp[temp['Event Date'].astype(int) < enddate]
    df_team_players = df_players.merge(df_teams, how='inner', left_on='Team', right_on='College')
    temp = temp.merge(df_team_players, how='left', left_on='TeamAPlayer1', right_on='Name')
    temp['TeamAPlayer1'] = temp['TeamAPlayer1'].apply(lambda x: x.strip())
    temp['TeamAPlayer2'] = temp['TeamAPlayer2'].apply(lambda x: x.strip())
    temp['TeamBPlayer1'] = temp['TeamBPlayer1'].apply(lambda x: x.strip())
    temp['TeamBPlayer2'] = temp['TeamBPlayer2'].apply(lambda x: x.strip())
    temp = temp[temp['Division'] == division].iloc[:,[0,1,2,3,4,5,6,7,8]].drop_duplicates()
    temp = temp.sort_values('Event Date', ascending=True)    
    return  temp

## Matchups > Prepare Matchup Data

In [10]:
def prepare_matchup_data(m_dict):
    s_new = s[0]
    d_new = s[1]
    m = {}

    temp = None
    for a in s_new:
        print(a,)
        for b in s_new[a]:
            if len(b) == 5:
                team1 =  b[1]
                team2 =  b[2]
                #print team1, team2, "\n"
            elif len(b) == 7:
                if team1 == (b[5].replace('(', '').replace(')', '')):
                    winner = team1
                    loser = team2
                else:
                    winner = team2
                    loser = team1

                m[a] = [b[0], b[1], b[2], b[3], b[4], winner, loser,a,  d_new[a]]
                #print a
                #print m
                df_ = pd.DataFrame.from_dict(m, orient='index')
                df_.columns=['Position','TeamAPlayer1', 'TeamAPlayer2', 'TeamBPlayer1', 'TeamBPlayer2',\
                                                                   'Winner', 'Loser', 'Compid', 'Event Date' ]

                temp = pd.concat([temp,df_], axis=0)
    return temp
#df_matchups = prepare_matchup_data(s)
#df_matchups.sort_values('Event Date', ascending=True)
#df_2018_matchups = select_matchups( 20180000, 20180900).sort_values('Event Date', ascending=True)
#df_2018_matchups.sort_values('Event Date', ascending=True).Compid.unique()

# Ratings
## Ratings > Set Defaults
### Ratings > Set Defaults > Player Ratings

In [11]:
def set_default_player_rating(df):
    df_temp = df.copy()
    df_temp['mu'] = Rating().mu
    df_temp['sigma'] = Rating().sigma
    
    df_temp = df_temp.reset_index()
    df_temp = df_temp.iloc[:, 1:]
    return df_temp

### Ratings > Lookup > Player Ratings

In [12]:
def lookup_player_rating(name, teams, df): 
    name = name.strip()
    teams = [a.strip() for a in teams]
    player_info =  df[(df['Name'] == name) & (df['Team'].isin(teams))].max()
    return player_info

## Questions on k-factor
Is a team really a position 2 team if they were only position 2 once in the season?
    - No, a team is a position 2 team if and only if they competed at position 2 for more than 50% of the season
Do you calibrate rankings based on number of games played?
    - The ranking algorithm does nominally rewards points for extra matches played to distance athletes that have competed againsts collegiate competition and folks that have never competed and start with a ranking of 25.


## Goals for k-factor
Normalize for number of games played?
Position 2 players should not exceed

## Expectations
I should see 3 normal curves with a phase shift in one direction between the five positions. 
    - The top quartile position 2 players should overlap ratings of the bottom quartile of the top position 1 players
    - The top quartile position 3 players should overlap ratings of the bottom quartile of the top position 2 players
    - The top quartile position 4 players should overlap ratings of the bottom quartile of the top position 3 players
    - The top quartile position 5 players should overlap ratings of the bottom quartile of the top position 4 players
    - The bottom position 5 players should overlap ratings of players who have not played in competition
    

## Ratings > Update
### Ratings > Update > Update Player Ratings

In [33]:
def update_player_ratings(matches, player_rating, players):
    hist = {}
    incr = 0
    k_factor = [1.00,1.00,1.00,1.00,1.00]
    matches = matches.sort_values('Event Date')
    matches = matches.reset_index().iloc[:,1:]
    #print matches
    for idx, rows in matches.iterrows():
        try:
            #print rows.iloc[1:5]
            p_lookup = {}
            r = {}
            t = {}
            my_index = {}
            for i in range(1,5):
                p_lookup[i] = lookup_player_rating(rows.iloc[1],[rows.iloc[5], rows.iloc[6]], player_rating )
                r[i] = Rating(mu=p_lookup[i].mu, sigma=p_lookup[i].sigma)
            print r

            t[1] = [r[1], r[2]]
            t[2] = [r[3], r[4]]

            my_ranks = [0,1] if p_lookup[1]['Team'] == rows['Winner'].strip() else [1,0]
            (r['1_new'], r['2_new']), (r['3_new'], r['4_new']) = rate([t[1], t[2]], ranks=my_ranks)

            for i in range(1,5):
                incr = incr+1
                my_index[i] = player_rating[player_rating['Name'] == rows.iloc[i]].index
                player_rating.loc[my_index[i], 'mu'] = r[str(i)+'_new'].mu #*k_factor[rows.iloc[0]-1]
                player_rating.loc[my_index[i], 'sigma'] = r[str(i)+'_new'].sigma
                hist[incr] = [rows.iloc[0], rows.iloc[i], rows.iloc[5],  rows.iloc[6], rows.iloc[7],rows.iloc[8],r[i].mu, r[str(i)+'_new'].mu] #*k_factor[rows.iloc[0]-1]]
        except:
            print rows


    return hist

df_player_rating = set_default_player_rating(df_players)
df_2018_matchups = select_matchups( 20180000, 20180900, division='DI')
df_2018_matchups['Event Date'] = df_2018_matchups['Event Date'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))
df_2018_matchups = df_2018_matchups.drop_duplicates()
df_2018_matchups = df_2018_matchups.sort_values(['Event Date','Compid'])
df_2018_matchups = df_2018_matchups.reset_index().iloc[:,1:]
df_2018_matchups
rating_history = update_player_ratings(df_2018_matchups, df_player_rating, df_players) 


{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: t

{1: trueskill.Rating(mu=21.892, sigma=7.774), 2: trueskill.Rating(mu=21.892, sigma=7.774), 3: trueskill.Rating(mu=21.892, sigma=7.774), 4: trueskill.Rating(mu=21.892, sigma=7.774)}
{1: trueskill.Rating(mu=21.892, sigma=7.774), 2: trueskill.Rating(mu=21.892, sigma=7.774), 3: trueskill.Rating(mu=21.892, sigma=7.774), 4: trueskill.Rating(mu=21.892, sigma=7.774)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: t

{1: trueskill.Rating(mu=21.892, sigma=7.774), 2: trueskill.Rating(mu=21.892, sigma=7.774), 3: trueskill.Rating(mu=21.892, sigma=7.774), 4: trueskill.Rating(mu=21.892, sigma=7.774)}
{1: trueskill.Rating(mu=21.892, sigma=7.774), 2: trueskill.Rating(mu=21.892, sigma=7.774), 3: trueskill.Rating(mu=21.892, sigma=7.774), 4: trueskill.Rating(mu=21.892, sigma=7.774)}
{1: trueskill.Rating(mu=21.892, sigma=7.774), 2: trueskill.Rating(mu=21.892, sigma=7.774), 3: trueskill.Rating(mu=21.892, sigma=7.774), 4: trueskill.Rating(mu=21.892, sigma=7.774)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: trueskill.Rating(mu=25.000, sigma=8.333), 4: trueskill.Rating(mu=25.000, sigma=8.333)}
{1: trueskill.Rating(mu=25.000, sigma=8.333), 2: trueskill.Rating(mu=25.000, sigma=8.333), 3: t

{1: trueskill.Rating(mu=27.886, sigma=6.809), 2: trueskill.Rating(mu=27.886, sigma=6.809), 3: trueskill.Rating(mu=27.886, sigma=6.809), 4: trueskill.Rating(mu=27.886, sigma=6.809)}
{1: trueskill.Rating(mu=33.615, sigma=6.809), 2: trueskill.Rating(mu=33.615, sigma=6.809), 3: trueskill.Rating(mu=33.615, sigma=6.809), 4: trueskill.Rating(mu=33.615, sigma=6.809)}
{1: trueskill.Rating(mu=22.601, sigma=6.809), 2: trueskill.Rating(mu=22.601, sigma=6.809), 3: trueskill.Rating(mu=22.601, sigma=6.809), 4: trueskill.Rating(mu=22.601, sigma=6.809)}
{1: trueskill.Rating(mu=27.886, sigma=6.809), 2: trueskill.Rating(mu=27.886, sigma=6.809), 3: trueskill.Rating(mu=27.886, sigma=6.809), 4: trueskill.Rating(mu=27.886, sigma=6.809)}
{1: trueskill.Rating(mu=22.601, sigma=6.809), 2: trueskill.Rating(mu=22.601, sigma=6.809), 3: trueskill.Rating(mu=22.601, sigma=6.809), 4: trueskill.Rating(mu=22.601, sigma=6.809)}
{1: trueskill.Rating(mu=30.973, sigma=7.268), 2: trueskill.Rating(mu=30.973, sigma=7.268), 3: t

{1: trueskill.Rating(mu=21.892, sigma=7.774), 2: trueskill.Rating(mu=21.892, sigma=7.774), 3: trueskill.Rating(mu=21.892, sigma=7.774), 4: trueskill.Rating(mu=21.892, sigma=7.774)}
{1: trueskill.Rating(mu=28.108, sigma=7.774), 2: trueskill.Rating(mu=28.108, sigma=7.774), 3: trueskill.Rating(mu=28.108, sigma=7.774), 4: trueskill.Rating(mu=28.108, sigma=7.774)}
{1: trueskill.Rating(mu=24.110, sigma=6.394), 2: trueskill.Rating(mu=24.110, sigma=6.394), 3: trueskill.Rating(mu=24.110, sigma=6.394), 4: trueskill.Rating(mu=24.110, sigma=6.394)}
{1: trueskill.Rating(mu=21.892, sigma=7.774), 2: trueskill.Rating(mu=21.892, sigma=7.774), 3: trueskill.Rating(mu=21.892, sigma=7.774), 4: trueskill.Rating(mu=21.892, sigma=7.774)}
{1: trueskill.Rating(mu=36.055, sigma=6.394), 2: trueskill.Rating(mu=36.055, sigma=6.394), 3: trueskill.Rating(mu=36.055, sigma=6.394), 4: trueskill.Rating(mu=36.055, sigma=6.394)}
{1: trueskill.Rating(mu=30.973, sigma=7.268), 2: trueskill.Rating(mu=30.973, sigma=7.268), 3: t

...

{1: trueskill.Rating(mu=37.187, sigma=2.475), 2: trueskill.Rating(mu=37.187, sigma=2.475), 3: trueskill.Rating(mu=37.187, sigma=2.475), 4: trueskill.Rating(mu=37.187, sigma=2.475)}
{1: trueskill.Rating(mu=40.253, sigma=2.160), 2: trueskill.Rating(mu=40.253, sigma=2.160), 3: trueskill.Rating(mu=40.253, sigma=2.160), 4: trueskill.Rating(mu=40.253, sigma=2.160)}
{1: trueskill.Rating(mu=36.812, sigma=2.124), 2: trueskill.Rating(mu=36.812, sigma=2.124), 3: trueskill.Rating(mu=36.812, sigma=2.124), 4: trueskill.Rating(mu=36.812, sigma=2.124)}
{1: trueskill.Rating(mu=42.817, sigma=2.089), 2: trueskill.Rating(mu=42.817, sigma=2.089), 3: trueskill.Rating(mu=42.817, sigma=2.089), 4: trueskill.Rating(mu=42.817, sigma=2.089)}
{1: trueskill.Rating(mu=36.107, sigma=2.239), 2: trueskill.Rating(mu=36.107, sigma=2.239), 3: trueskill.Rating(mu=36.107, sigma=2.239), 4: trueskill.Rating(mu=36.107, sigma=2.239)}
{1: trueskill.Rating(mu=41.950, sigma=2.124), 2: trueskill.Rating(mu=41.950, sigma=2.124), 3: t

{1: trueskill.Rating(mu=39.269, sigma=1.964), 2: trueskill.Rating(mu=39.269, sigma=1.964), 3: trueskill.Rating(mu=39.269, sigma=1.964), 4: trueskill.Rating(mu=39.269, sigma=1.964)}
{1: trueskill.Rating(mu=26.600, sigma=3.576), 2: trueskill.Rating(mu=26.600, sigma=3.576), 3: trueskill.Rating(mu=26.600, sigma=3.576), 4: trueskill.Rating(mu=26.600, sigma=3.576)}
{1: trueskill.Rating(mu=26.942, sigma=4.602), 2: trueskill.Rating(mu=26.942, sigma=4.602), 3: trueskill.Rating(mu=26.942, sigma=4.602), 4: trueskill.Rating(mu=26.942, sigma=4.602)}
{1: trueskill.Rating(mu=41.635, sigma=1.964), 2: trueskill.Rating(mu=41.635, sigma=1.964), 3: trueskill.Rating(mu=41.635, sigma=1.964), 4: trueskill.Rating(mu=41.635, sigma=1.964)}
{1: trueskill.Rating(mu=45.251, sigma=2.239), 2: trueskill.Rating(mu=45.251, sigma=2.239), 3: trueskill.Rating(mu=45.251, sigma=2.239), 4: trueskill.Rating(mu=45.251, sigma=2.239)}
{1: trueskill.Rating(mu=48.981, sigma=2.282), 2: trueskill.Rating(mu=48.981, sigma=2.282), 3: t

{1: trueskill.Rating(mu=49.218, sigma=1.964), 2: trueskill.Rating(mu=49.218, sigma=1.964), 3: trueskill.Rating(mu=49.218, sigma=1.964), 4: trueskill.Rating(mu=49.218, sigma=1.964)}
{1: trueskill.Rating(mu=37.215, sigma=2.160), 2: trueskill.Rating(mu=37.215, sigma=2.160), 3: trueskill.Rating(mu=37.215, sigma=2.160), 4: trueskill.Rating(mu=37.215, sigma=2.160)}
{1: trueskill.Rating(mu=25.505, sigma=2.199), 2: trueskill.Rating(mu=25.505, sigma=2.199), 3: trueskill.Rating(mu=25.505, sigma=2.199), 4: trueskill.Rating(mu=25.505, sigma=2.199)}
{1: trueskill.Rating(mu=34.127, sigma=3.126), 2: trueskill.Rating(mu=34.127, sigma=3.126), 3: trueskill.Rating(mu=34.127, sigma=3.126), 4: trueskill.Rating(mu=34.127, sigma=3.126)}
{1: trueskill.Rating(mu=43.294, sigma=2.124), 2: trueskill.Rating(mu=43.294, sigma=2.124), 3: trueskill.Rating(mu=43.294, sigma=2.124), 4: trueskill.Rating(mu=43.294, sigma=2.124)}
{1: trueskill.Rating(mu=46.951, sigma=1.909), 2: trueskill.Rating(mu=46.951, sigma=1.909), 3: t

{1: trueskill.Rating(mu=39.839, sigma=2.160), 2: trueskill.Rating(mu=39.839, sigma=2.160), 3: trueskill.Rating(mu=39.839, sigma=2.160), 4: trueskill.Rating(mu=39.839, sigma=2.160)}
{1: trueskill.Rating(mu=42.261, sigma=2.160), 2: trueskill.Rating(mu=42.261, sigma=2.160), 3: trueskill.Rating(mu=42.261, sigma=2.160), 4: trueskill.Rating(mu=42.261, sigma=2.160)}
{1: trueskill.Rating(mu=43.859, sigma=2.089), 2: trueskill.Rating(mu=43.859, sigma=2.089), 3: trueskill.Rating(mu=43.859, sigma=2.089), 4: trueskill.Rating(mu=43.859, sigma=2.089)}
{1: trueskill.Rating(mu=36.641, sigma=3.033), 2: trueskill.Rating(mu=36.641, sigma=3.033), 3: trueskill.Rating(mu=36.641, sigma=3.033), 4: trueskill.Rating(mu=36.641, sigma=3.033)}
{1: trueskill.Rating(mu=42.705, sigma=2.124), 2: trueskill.Rating(mu=42.705, sigma=2.124), 3: trueskill.Rating(mu=42.705, sigma=2.124), 4: trueskill.Rating(mu=42.705, sigma=2.124)}
{1: trueskill.Rating(mu=34.089, sigma=1.964), 2: trueskill.Rating(mu=34.089, sigma=1.964), 3: t

{1: trueskill.Rating(mu=45.842, sigma=1.936), 2: trueskill.Rating(mu=45.842, sigma=1.936), 3: trueskill.Rating(mu=45.842, sigma=1.936), 4: trueskill.Rating(mu=45.842, sigma=1.936)}
{1: trueskill.Rating(mu=47.655, sigma=1.858), 2: trueskill.Rating(mu=47.655, sigma=1.858), 3: trueskill.Rating(mu=47.655, sigma=1.858), 4: trueskill.Rating(mu=47.655, sigma=1.858)}
{1: trueskill.Rating(mu=42.485, sigma=1.936), 2: trueskill.Rating(mu=42.485, sigma=1.936), 3: trueskill.Rating(mu=42.485, sigma=1.936), 4: trueskill.Rating(mu=42.485, sigma=1.936)}
{1: trueskill.Rating(mu=46.210, sigma=1.883), 2: trueskill.Rating(mu=46.210, sigma=1.883), 3: trueskill.Rating(mu=46.210, sigma=1.883), 4: trueskill.Rating(mu=46.210, sigma=1.883)}
{1: trueskill.Rating(mu=34.810, sigma=1.909), 2: trueskill.Rating(mu=34.810, sigma=1.909), 3: trueskill.Rating(mu=34.810, sigma=1.909), 4: trueskill.Rating(mu=34.810, sigma=1.909)}
{1: trueskill.Rating(mu=50.287, sigma=1.883), 2: trueskill.Rating(mu=50.287, sigma=1.883), 3: t

In [39]:
#rating_history = pd.DataFrame.from_dict(rating_history, orient='index')
#rating_history.columns=['Position', 'Name', 'Winning Team', 'Losing Team', 'Compid', 'Event Date', 'Rating Before Match', 'Rating After Match']
#rating_history.to_csv("calcdratings.csv")
rating_history.sort_values('Rating Before Match', ascending=False)

,Position,Name,Winning Team,Losing Team,Compid,Event Date,Rating Before Match,Rating After Match
8887,2,Desiree Sukhov,Cal Poly SLO,Cal State Bakersfield,1437,2018-03-31,55.835298,54.888284
8885,2,Emily Sonny,Cal Poly SLO,Cal State Bakersfield,1437,2018-03-31,55.835298,56.782312
8888,2,Sarah Garrow,Cal Poly SLO,Cal State Bakersfield,1437,2018-03-31,55.835298,54.888284
8886,2,Sam Manley,Cal Poly SLO,Cal State Bakersfield,1437,2018-03-31,55.835298,56.782312
8492,2,Nickeisha Williams,Cal Poly SLO,Cal State Northridge,1361,2018-03-30,54.834081,53.832864
8489,2,Emily Sonny,Cal Poly SLO,Cal State Northridge,1361,2018-03-30,54.834081,55.835298
8490,2,Sam Manley,Cal Poly SLO,Cal State Northridge,1361,2018-03-30,54.834081,55.835298
8491,2,Kristey Markle,Cal Poly SLO,Cal State Northridge,1361,2018-03-30,54.834081,53.832864
8065,2,Sam Manley,Cal Poly SLO,"Cal State University, Sacramento",1386,2018-03-30,53.773624,54.834081
8068,2,Lana Brown,Cal Poly SLO,"Cal State University, Sacramento",1386,2018-03-30,53.773624,52.713167


In [ ]:
df_player_rating.sort_values('mu', ascending=False).head(50)


In [ ]:
test = df_rating_history[df_rating_history['Name'].isin(['Megan McNamara'])].sort_values(['Event Date'], ascending=True)
test['delta'] = test['Rating After Match'] - test['Rating Before Match']
test

In [ ]:
test = df_rating_history[df_rating_history['Name'].isin(['Mima Mirkovic'])].sort_values(['Event Date'], ascending=True)
test['delta'] = test['Rating After Match'] - test['Rating Before Match']
test

In [ ]:

df_plot = df_rating_history[['Name', 'Event Date', 'Rating After Match', 'Position' ]]
pl = sns.lineplot(x="Event Date", y="Rating After Match",  hue='Position', markers=True,  data=df_plot)


In [ ]:
def show_individual_rating(player):
    df_plot = df_rating_history[df_rating_history.Name.isin(player)]
    df = df_rating_history[df_rating_history.Name.isin(player)]
    df_plot = df_plot[['Name', 'Event Date', 'Rating After Match', 'Position' ]]
    a4_dims = (15, 8)
    fig, ax = pyplot.subplots(figsize=a4_dims)
    g = sns.lineplot(x="Event Date", y="Rating After Match", hue='Name', markers=True, style='Position', data=df_plot, ax=ax)
    return df

top = list(df_player_rating.sort_values('mu', ascending=False).head(2)['Name'])
show_individual_rating(['Emily Maglio'])

In [ ]:
def showRatingByTeam(school):
    df_plot = df_rating_history[df_rating_history.Name.isin(list(df_players[df_players['Team'] == school]['Name']))]
    df_plot = df_plot[['Name', 'Event Date', 'Rating After Match', 'Position' ]]
    a4_dims = (20.7, 8.27)
    fig, ax = pyplot.subplots(figsize=a4_dims)
    sns.lineplot(x="Event Date", y="Rating After Match",  hue='Name', markers=True,  data=df_plot, ax=ax)
    g = df_rating_history[df_rating_history.Name.isin(list(df_players[df_players['Team'] == school]['Name']))]
    return g

In [ ]:
https://www.microsoft.com/en-us/research/project/trueskill-ranking-system/?from=http%3A%2F%2Fresearch.microsoft.com%2Fen-us%2Fprojects%2Ftrueskill%2F

# D1 Matchups

In [ ]:
df_player_rating = set_default_player_rating(df_players)
df_2018_matchups = df_matchups[df_matchups['Event Date'].astype(int) > 20180000].reset_index()
df_2018_matchups = df_2018_matchups.iloc[:,1:]
d1_colleges = list(df_teams[df_teams['Division'] == 'DI'].College.unique())

df_2018_d1_matchups = df_2018_matchups[df_2018_matchups['Winner'].isin(d1_colleges)].reset_index()
df_2018_d1_matchups = df_2018_d1_matchups.iloc[:,1:]

In [ ]:
d = df_rating_history[df_rating_history.Position == 1]['Rating After Match']
a4_dims = (20, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.distplot(d); #28

In [ ]:
d = df_rating_history[df_rating_history.Position == 2]['Rating After Match']
a4_dims = (20, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.distplot(d); #28

In [ ]:
d = df_rating_history[df_rating_history.Position == 3]['Rating After Match']
a4_dims = (20, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.distplot(d); #28

In [ ]:
d = df_rating_history[df_rating_history.Position == 4]['Rating After Match']
a4_dims = (20, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.distplot(d); #27

In [ ]:
d = df_rating_history[df_rating_history.Position == 5]['Rating After Match']
a4_dims = (20, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.distplot(d); #30

In [ ]:
d = df_rating_history['Rating After Match']
a4_dims = (20, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.distplot(d); #28

## D1 Matchups > All Players

In [ ]:
df_player_rating = set_default_player_rating(df_players)
update_player_ratings(df_2018_d1_matchups, df_player_rating, df_players)
df_player_rating.sort_values('mu', ascending=False)
df_player_rating.sort_values('mu', ascending=False).head(22)

## D1 Matchups By Position

In [ ]:
def getTopPlayersByPosition(position):
    player_rating_ = set_default_player_rating(df_players)
    matchups_ = select_matchups( 20180000, 20180900, division='DI').sort_values(['Event Date', 'Compid', 'Position'], ascending=True).drop_duplicates().reset_index().iloc[:,1:]
    if position != 'all':
        matchups_ = matchups_[matchups_['Position'].astype(int) == position]
    df_rating_ = update_player_ratings(matchups_, player_rating_, df_players)
    fig, ax = pyplot.subplots(figsize=a4_dims)
    g = sns.distplot(player_rating_['mu'][player_rating_['mu']>30])
    g.set(xlim=(25, 80))
    return player_rating_.sort_values('mu', ascending=False), df_rating_
df_all = getTopPlayersByPosition('all') 


In [ ]:
df_positions = pd.DataFrame.from_dict(df_all[1]).T
df_positions.columns=['Position', 'Name', 'Winning Team', 'Losing Team', 'Compid', 'Event Date', 'Rating Before Match', 'Rating After Match']
df_positions.to_csv("calcdratings.csv")

In [ ]:
df_1 = getTopPlayersByPosition(1) 

In [ ]:
df_2 = getTopPlayersByPosition(2) 

In [ ]:
df_3 = getTopPlayersByPosition(3) 

In [ ]:
df_4 = getTopPlayersByPosition(4) 

In [ ]:
df_5 = getTopPlayersByPosition(5) 

# Probability and Statistics

In [ ]:
def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = global_env()
    return ts.cdf(delta_mu / denom)

In [ ]:
def actual_match(compid, df):
    return df[df['Compid'].astype(int) == compid].groupby('Winner')['Position'].count()

actual_match(1863, df_matchups)

In [ ]:
def predict_match(compid, df):
    match = df[df['Compid'].astype(int) == compid].copy().reset_index().iloc[:,1:]
    for idx, rows in match.iterrows():
        r = {}
        p_lookup = {}
        t = {}
        for i in range(1,5):
            p_lookup[i] = lookup_player_rating(rows.iloc[i],[rows.iloc[5], rows.iloc[6]], df_player_rating )
            r[i] = Rating(mu=p_lookup[i].mu, sigma=p_lookup[i].sigma)
            #match[i] = r[i]
    
        t[1] = [r[1], r[2]]
        t[2] = [r[3], r[4]]
        
        win_prob = win_probability(t[1], t[2])
    
        
        match.loc[idx, 'Probability'] = win_prob 
        #print win_prob
        if (win_prob >.60):
            match.loc[idx, 'Predicted Winner'] = p_lookup[1]['Team'] 
            match.loc[idx, 'Distance From Draw'] = (win_prob - .60)
            match.loc[idx, 'T1 Win'] = 1
        elif (win_prob <.40):
            match.loc[idx, 'Predicted Winner'] = p_lookup[3]['Team'] 
            match.loc[idx, 'Distance From Draw'] = (.40-win_prob )
            match.loc[idx, 'T1 Win'] = 0
        else:
            match.loc[idx, 'Predicted Winner'] = 'Draw'
            match.loc[idx, 'Distance From Draw'] = 0
            match.loc[idx, 'T1 Win'] = .5
        
    
    return match

_df = {}
temp = None
for a in range(780, 1870):
    print a,
    _df = predict_match(a, df_matchups)
    _df = pd.concat([_df, temp], axis=0)
    temp = _df


total_no_draw = len(_df[_df['Winner'] != 'Draw'])
accurate = len(_df[_df['Winner'] == _df['Predicted Winner']])
print accurate, total_no_draw, accurate*100/total_no_draw

_df

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
y_true = list(_df['T1 Win'])
y_pred = list(_df['Probability'])
accuracy_score(y_true, y_pred)

#accuracy_score(y_true, y_pred, normalize=False)


In [ ]:
_df = _df.reset_index()
_df = _df.iloc[:,1:]
_df['Actual'] = 0
idx = _df.index[_df['Winner'] == _df['Predicted Winner']]
#_df.index = idx, 'Actual'] = 1
#df.Actual.where[_df['Winner'] == _df['Predicted Winner']])

_df['Actual'] = np.where(_df['Winner'] == _df['Predicted Winner'], 1, 0)
_df

In [ ]:
new = _df[['Predicted Winner', 'Winner', 'Loser', 'Probability', 'Distance From Draw']].sort_values('Distance From Draw', ascending=False)
for idx, rows in new.iterrows():
    print rows[0],rows[1],  rows[4]

## 2019 Potential (Look for Pre-season Rankings)
Takes a look at team without last year's seniors

In [ ]:
df_2019_Potential = df_player_rating[~df_player_rating['Year'].isin(['Sr','R-Sr', 'GS'])].sort_values('mu', ascending=False)
df_2019_Potential = df_2019_Potential.groupby('Team')['mu'].nlargest(8).sum(level=0).sort_values(ascending=False).to_frame()
df_2019_Potential.columns = ['Total K Score']
df_2019_Potential
#filter out D1s

In [ ]:
#Strength of schedule?

## 2018 Post Season Team Strength Ranking By Top 10 Cumulative Player Score

In [ ]:
df_2018_final = df_player_rating.sort_values('mu', ascending=False)
df_2018_final = df_2018_final.groupby('Team')['mu'].nlargest(10).sum(level=0).sort_values(ascending=False).to_frame()
df_2018_final.columns = ['Total K Score']
df_2018_final

In [ ]:
def predictMatch(_compid, _df):
    _match = df_all_matchups[df_all_matchups['CompID'] == _compid].iloc[:,0:5]
    wins = {}

    for i in range(0,len(_match)):
        r = [Rating(lookupPlayerRating(_match.iloc[i,j], predict_ranking))  for j in range(1,5)]

        t1 = [r[0], r[1]]
        t2 = [r[2], r[3]]

        win_prob = win_probability(t1, t2)

        #print [(_match.iloc[i,j]+"("+lookupPlayer(_match.iloc[i,j], _df)['Team']+")") for j in range(1,5) ],

        if win_prob > .50:
            val = max(lookupPlayer(_match.iloc[i, 1], _df)['Team'], lookupPlayer(_match.iloc[i, 2], _df)['Team'])
            #print val, ' favored'
            try:
                wins[val] += 1
            except:
                wins[val] =  1
        elif win_prob < .50:
            val = max(lookupPlayer(_match.iloc[i, 3], _df)['Team'], lookupPlayer(_match.iloc[i, 4], _df)['Team'])
            #print val, ' favored'
            try:
                wins[val] += 1
            except:
                wins[val] =  1
        else:
            val = 'Draw'
            try:
                wins[val] += 1
            except:
                wins[val] =  1
                
    match_winner = max(wins.iteritems(), key=operator.itemgetter(1))[0]      
    
    return wins, match_winner

print "Predicted:" + str(predictMatch(1816, df_player_ratings))
print "Actual:" + str(actualMatch(1816, df_player_ratings))

# Pairs Rating
I shall assert that a pairs rating is more than the sum of ratings of players. In additional to 

In [ ]:
df_2018_matchups
t1 = df_2018_matchups[['TeamAPlayer1', 'TeamAPlayer2']].copy()
t2 = df_2018_matchups[['TeamBPlayer1', 'TeamBPlayer2']].copy()
t1.columns = ['P1', 'P2']
t2.columns = ['P1', 'P2']
df_pairs = pd.concat([t1,t2], axis=0)
df_pairs = pd.merge(df_pairs, df_player_ratings[['Player','Team']], how='inner', left_on='P1', right_on='Player')[['P1', 'P2', 'Team']]
df_pairs.drop_duplicates(inplace=True)
df_pairs

# Team Rating
N:M unbalanced match – [(r1,), (r2, r3, r4)]

# Ranking

In [ ]:
def get_positional_ranking(startdate, enddate, position, division ='DI' ):
    df_pr = setDefaultPlayerRating(df_players)
    new_df = select_matchups( 20180000, 20180900)

    df_player_ratings_new = updatePlayerRatings(_new_df, _df_player_rating)
    
    top_players = df_player_ratings_new.sort_values('mu', ascending=False).reset_index()
    top_players = top_players.iloc[:,1:]
    top_players['Rank'] = top_players.index+1
    return top_players

#pos1_ranking = getPositionalRanking('DI', 20180000, 20190000, 1, df_all_players)
#pos2_ranking = getPositionalRanking('DI', 20180000, 20190000, 2, df_all_players)
current_ranking = getPositionalRanking('DI', 2018000, 20180600, 'all', df_all_players)
current_ranking

In [ ]:
current_ranking['Year'].unique()
#current_ranking[current_ranking['Year'] == None]
curr = current_ranking[current_ranking['Year'] != '']
curr = curr[~curr['Year'].isnull() ]
curr = curr[~curr['Year'].str.contains('Sr')]
curr.iloc[:60,:].groupby(['Team'])['mu'].sum().sort_values(ascending=False).to_frame()

# Pre-season rankings

In [ ]:
current_ranking['Year'].unique()
current_ranking[current_ranking['Year'] == None]
curr = current_ranking[current_ranking['Year'] != '']
curr = curr[~curr['Year'].isnull() ]
#curr = curr[~curr['Year'].str.contains('Sr')]
curr.iloc[:60,:].groupby(['Team'])['mu'].sum().sort_values(ascending=False).to_frame()


In [ ]:
def lookup_player_rating(name, df):
    v_idx = max(_df.index[_df.Player.str.contains(_name.lstrip())])
    v_idx = max(_df.index[_df.Player.str.contains(_name.rstrip())])
    v = _df.loc[v_idx]
    return v['mu'], v['sigma']

In [ ]:
for i in range(1816,1817):
    print  predictMatch(i, df_player_ratings)[1], 'Predict',
    print   actualMatch(i, df_player_ratings)[1], 'Actual',
    try:
        if predictMatch(i, df_player_ratings)[1] == actualMatch(i, df_player_ratings)[1]:
            print i, "Correct"
        else:
            print i, "Wrong"
    except:
        print i, "No Data"

In [ ]:
df_all_matchups[df_all_matchups['CompID'] == 1802]

In [ ]:

def lookupPlayer(_name, _df):
    #print _name
    v_idx = _df.index[_df.Player==_name]
    v = _df.iloc[v_idx]
    return v_idx #_df

lookupPlayer(_name, df_all_players)
df_all_players[df_all_players['Team'] == 'Sierra College']  #.unique()

# Utility

In [ ]:
def loadTable(_url, _i, _type, _columns):
    soup, table = scrapeUrl(_url, _i)
    my_labels = table[0]
    my_results = table[1:]
    
    if (_type == 'Team') & (my_labels[2] != ''):
        #for loading Player tables
        try:
            df = pd.DataFrame.from_records(my_results, columns=_columns[:4])
            df[_type] = getHeader(soup, 'College Players for ', 'player')
            df[_type + ' ID'] = _i
        except:
            try:
                df = pd.DataFrame.from_records(my_results, columns=my_labels)
                df[_type] = getHeader(soup, 'College Players for ', 'player')
                df[_type + ' ID'] = _i
            except:
                df = None
    
    
    if ((_type == 'Matchup') & (my_labels[1] != '')) > 0 :
        my_labels.insert(2, my_labels[2])
        my_labels.insert(1, my_labels[1])
        #print my_labels
        df = pd.DataFrame.from_records(my_results, columns=my_labels) 
        df[_type] = getHeader(soup, 'Competition on ', 'event')
        df[_type + ' ID'] = _i

        try:
            df['Location'] = getTag(soup, 'Location: ', 'p')
        except:
            df['Location'] = None
            
        try:
            #df['Winner'] = df['Winner'].apply(lambda x: re.sub(r'[\(\)]', '', x))
            df['Winner'] = df['Winner'].str.replace('\(', '')
            df['Winner'] = df['Winner'].str.replace('\)', '')
        except:
            df['Winner'] = None
            
        
            
        df = df[~df.iloc[:,1].str.contains("Game")]
        df = df[~df.iloc[:,3].str.contains("forfeit")]
        df = df[~df.iloc[:,4].str.contains("0-0")]
    else:
        df = None
        
    return df


def mergeResults(_url, _start, _end, _type, _columns):
    d = {i:loadTable(_url, i, _type, _columns) for i in range(_start, _end)}
    for k in d.keys():
        if d[k] is not None:
            d[k].columns = _columns 
    return pd.concat(d.values(), axis=0)

getMatchups(1802, 1803)

In [ ]:
### OLD VERSION

from datetime import datetime

def scrape_matchups(start=match_start_id, end=match_end_id):
    url = 'http://collegebeachvb.com/scores/competition/'

    d = {}
    
    for i in range(start, end):
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('Competition on ','')
        header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')
        
        if len(table[1:]) > 0:
            if len(table[1:][0]) == 7:
                my_labels = table[0]
                col_numbers = len(table[1:][0]) + 2
                my_results = table[1:]
                
                #print my_labels
                
                d[i] = pd.DataFrame.from_records(my_results)
                d[i]['Event Date'] = header

                #try:
                d[i][5] = d[i][5].str.replace('\(', '')
                d[i][5] = d[i][5].str.replace('\)', '')
                #except:
                #    d[i][5] = None

                d[i] = d[i][~d[i].iloc[:,1].str.contains("Game", "forfeit")]
                d[i] = d[i][~d[i].iloc[:,3].str.contains("forfeit")]
                d[i] = d[i][~d[i].iloc[:,4].str.contains("0-0")]
                d[i]['compid'] = i
        else:

            print table
            
    temp = pd.concat([d[k] for k in d.keys()], axis=0)
    temp.columns = ['Position', 'TeamAPlayer1', 'TeamAPlayer2', 'TeamBPlayer1', 'TeamBPlayer2', 'Winner', 'Score', 'Event Date', 'Compid']
        
    return temp

#df_matchups = scrape_matchups(1, 1900)

In [ ]:
### OLD
def scrape_win_lose(start=match_start_id, end=match_end_id):
    url = 'http://collegebeachvb.com/scores/competition/'
    d = {}
    
    for i in range(start, end):
        print i,":",
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('Competition on ','')
        header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')
        
        my_labels = table[0]
        my_results = [a for a in table[1:] if a[1] not in 'Game was unfinished']
        
        d[i] = pd.DataFrame.from_records(my_results)
        d[i]['Event Date'] = header
        
        
        if len(my_labels) < 5:
            d[i][5] = d[i][5].str.replace('\(', '')
            d[i][5] = d[i][5].str.replace('\)', '')
            d[i]['Winner'] = d[i][5]
            try:
                d[i]['Loser'] = d[i]['Winner'].apply(lambda x:  my_labels[2] if x in my_labels[1] else my_labels[1])
            except:
                d[i]['Loser'] = None

        d[i]['Compid'] = i

        

  
        temp = pd.concat([d[k] for k in d.keys()], axis=0)
        #temp = temp[[0, 'Event Date', 'Compid', 'Winner', 'Loser']]
        #temp.columns = ['Position', 'Event Date', 'Compid', 'Winner', 'Loser']


    return temp
df_winner_loser = scrape_win_lose(713,1900)
df_winner_loser

In [ ]:
OLD
def scrape_matchups(start=match_start_id, end=match_end_id):
    url = 'http://collegebeachvb.com/scores/competition/'
    d = {}
    temp = None
    for i in range(start, end):
        print i,":",
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('Competition on ','')
        header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')

        my_labels = table[0]
        my_results = [a for a in table[1:] if a[1] not in ['Game was unfinished', '', None, 'Forfeit']]

        d[i] = pd.DataFrame.from_records(my_results)
        d[i]['Event Date'] = header

        if ((len(my_results) > 0) & (len(my_labels) == 5)):
            d[i][5] = d[i][5].str.replace('\(', '')
            d[i][5] = d[i][5].str.replace('\)', '')
            d[i]['Winner'] = d[i][5]
            try:
                d[i]['Loser'] = d[i]['Winner'].apply(lambda x:  my_labels[2] if x in my_labels[1] else my_labels[1])
            except:
                d[i]['Loser'] = None

            d[i]['Compid'] = i

            temp = pd.concat([d[k] for k in d.keys()], axis=0)
            temp = temp.rename(columns ={0: 'Position', 1: 'TeamAPlayer1', 2: 'TeamAPlayer2', 3:'TeamBPlayer1',\
                                         4:'TeamBPlayer2', 5:'Winner', 6:'Score'})
        else:
            print "error:", i
            
    return temp
df_matchups = scrape_matchups(780, 1900)
df_matchups

In [ ]:
def lookup_winner_loser(compid, position, df_=df_matchups):
    print compid, position
    temp =  df_[(df_.Compid.astype(int) == int(compid)) & (df_.Position.astype(int) == int(position))]
    return temp.Winner.item(), temp.Loser.item()
#& (df_.Position == position))

lookup_winner_loser(1024, 3,  df_2018_matchups)

In [ ]:

## OLD
def update_player_ratings(matches, player_ratings, players):
    for idx, rows in matches.iterrows():
        t = {}
        r = {}
        
        for i in range(1,5):
            try:
                r[str(i)+'_mu'], r[str(i)+'_sigma']  = lookup_player_rating(rows.iloc[i], player_ratings )
                r[i] = Rating(mu=r[str(i)+'_mu'], sigma=r[str(i)+'_sigma'])
                print r
            except:
                print "" #hi", rows

            
        t[1] = [r[1], r[2]]
        t[2] = [r[3], r[4]]
        

        p1_team = lookup_player(rows.iloc[1], players)

        if p1_team.strip()  == rows['Winner'].strip():
            my_ranks = [0,1]
        else:
            my_ranks = [1,0]

        (r['new_1'], r['new_2']), (r['new_3'], r['new_4']) = rate([t[1], t[2]], ranks=my_ranks)

        #print idx
        for i in range(1,5):
            #print i,
            try:
                mu, sigma = lookup_player_rating(rows.iloc[i], player_ratings )
                my_idx =  player_ratings.index[player_ratings['Name'] == rows.iloc[i:i+1].item()]
                player_ratings.loc[my_idx, 'mu'] = r['new_'+str(i)].mu
                player_ratings.loc[my_idx, 'sigma'] = r['new_'+str(i)].sigma
            except:
                print "" #mu, sigma, rows.iloc[i:i+1]
        
        #print "\n"
    return player_ratings


df_2018_matchups = select_matchups( 20180000, 20180900).sort_values('Event Date', ascending=True)
update_player_ratings(df_2018_matchups, df_player_ratings, df_players)
df_player_rating

In [ ]:
#OLD  
    
    df_player_rating = set_default_player_rating(df_players)
    df_2018_matchups = select_matchups( 20180000, 20180900, division='DI').sort_values(['Event Date', 'Compid', 'Position'], ascending=True).drop_duplicates()
    df_2018_matchups = df_2018_matchups.reset_index().iloc[:,1:]
    rating_history = update_player_ratings(df_2018_matchups, df_player_rating, df_players, k_factor) 
    rating_history
    df_plot = df_rating_history[['Name', 'Event Date', 'Rating After Match', 'Position' ]]
    a4_dims = (20.7, 8.27)
    fig, ax = pyplot.subplots(figsize=a4_dims)
    sns.lineplot(x="Event Date", y="Rating After Match",  hue='Position', markers=True,  data=df_plot, ax=ax)

In [ ]:
# This is only for 2018 rosters

def scrape_player_info(start= player_start_id, end= player_end_id):
    url = 'http://collegebeachvb.com/players/collegeplayers/'
    d = {}
    temp = None
    for i in range(start, end):
        print(i,)
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('College Players for ','')
        if (len(table) > 1) :
            if (len(table[1:][1]) == 4): 
                my_results = table[1:]
                my_labels = table[0]
                col_numbers = len(table[1:][0])
                my_results = table[1:]

                d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
                d[i]['Team'] = header
                d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
                #print d
            elif (len(table[1:][1]) == 1): 
                d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
                d[i]['Team'] = header
                d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
            elif (len(table[1:][1]) == 2): 
                d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
                d[i]['Team'] = header
                d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
            else:
                print(i, len(table[1:][1]))
                d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
                d[i]['Team'] = header
                d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
            
        temp = pd.concat([d[k] for k in d.keys()], axis=0)
                
    return temp

#df_players = scrape_player_info()
#df_players

df_players = pd.read_csv('Data/beachvb_players.csv').iloc[:,1:]
df_players['Name'] = df_players['Name'].apply(lambda x: x.strip())
df_players['Team'] = df_players['Team'].apply(lambda x: x.strip())
df_players = df_players[~df_players['Name'].str.contains('orfeit')]
df_players = df_players.drop(df_players.index[df_players['Name'].duplicated(keep='first')])

df_players.describe()